# Exploring the German-Korean Tatoeba dataset
[Github](https://github.com/Helsinki-NLP/Tatoeba-Challenge)

[Huggingface](https://huggingface.co/datasets/Helsinki-NLP/tatoeba_mt)

In [ ]:
from tatoeba import preprocess, analysis
import re

In [ ]:
dataset = preprocess.get_dataset()


In [3]:
subtitle_data = preprocess.get_subtitle_dataset()

In [4]:
len(subtitle_data)
print(subtitle_data)


Dataset({
    features: ['id', 'source', 'target'],
    num_rows: 996525
})


In [7]:
subtitle_data.to_csv("../data/subtitle_data.csv", num_proc=8)

Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

158408241

In [6]:
one_word_trg = analysis.get_one_word_sentences(subtitle_data)
print(one_word_trg)

#0:   0%|          | 0/125 [00:00<?, ?ba/s]

#1:   0%|          | 0/125 [00:00<?, ?ba/s]

#2:   0%|          | 0/125 [00:00<?, ?ba/s]

#3:   0%|          | 0/125 [00:00<?, ?ba/s]

#6:   0%|          | 0/125 [00:00<?, ?ba/s]

#5:   0%|          | 0/125 [00:00<?, ?ba/s]

#4:   0%|          | 0/125 [00:00<?, ?ba/s]

#7:   0%|          | 0/125 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'source', 'target'],
    num_rows: 79426
})


In [5]:
dat = subtitle_data.to_pandas()

In [ ]:
subsets = dataset.unique("id")


In [ ]:
print(subsets)
#OpenSubtitles-v2018
#TED2020-v1

#maybe also GlobalVoices-v2018q4 and Mozilla-I10n-v1